## 입력받아 동작하는 웹툰 정보 프로그램 만들기

### 웹툰 크롤러 ##
1. 정보
2. 에피소드 목록
0. 나가기

```> 입력: 1```

```> 웹툰명을 입력하세요: 유```

### 정보를 가져올 웹툰선택
1. 유일무이 로맨스
2. 윌유메리미
3. 유미의 세포들
4. 당신에게 끌리는 올바른 이유
...

```0. 이전```

```> 웹툰 선택: 3```

### 정보 (유미의 세포들)
작가: 이동건
설명: 어쩌고저쩌고....
장르: a, b, c, d

### 웹툰 크롤러 ##
1. 정보
2. 에피소드 목록
0. 나가기

```> 입력: 2```

```> 웹툰명을 입력하세요: 유```

### 에피소드 목록을 가져올 웹툰선택
1. 유일무이 로맨스
2. 윌유메리미
3. 유미의 세포들
4. 당신에게 끌리는 올바른 이유
...

```0. 이전```

```> 웹툰 선택: 3```

### 에피소드 목록 (유미의 세포들)
437화 당신이 받고 있는 시그널 5
 평점: 5.17, 날짜: 2019.12.03
436화 당신이 받고 있는 시그널 4
 평점: 9.58, 날짜: 2019.11.29
...
...
...
...

### 웹툰 크롤러 ##
1. 정보
2. 에피소드 목록
0. 나가기

```입력: 0```

In [ ]:
import requests
import re
from bs4 import BeautifulSoup

url = 'https://comic.naver.com/webtoon/weekday.nhn'
response = requests.get(url)
response.status_code
soup = BeautifulSoup(response.text)

def menu():
    print('''<<웹툰 크롤러>>

    1. 정보
    2. 에피소드 목록
    3. 나가기
    ''')

    value = input('입력: ')
    menu_move(value)

# 메뉴 이동
def menu_move(value):
    if value == '1':
        webtoon_name = input('\n웹툰명을 입력하세요: ')
        print('\n<<<정보를 가져올 웹툰 선택>>>')
        webtoon(value, webtoon_name)
        menu()
            
    elif value == '2':
        webtoon_name = input('\n웹툰명을 입력하세요: ')
        print('\n<<<에피소드를 가져올 웹툰 선택>>>')
        webtoon(value, webtoon_name)

    elif value == '3':
        print('크롤링을 종료합니다.')
        
    else:
        print('\n잘못 입력하셨습니다. 다시 입력하세요. \n')
        value = input('입력: ')
        menu_move(value)

# 웹툰 이름 찾기
def webtoon(menu_value, webtoon_name):
    a_list = soup.select('a.title[title*="{}"]'.format(webtoon_name))
    
    # 결과 dict를 저장할 리스트
    results = []
    
    # select로 선택한 a태그 목록을 순회
    for a in a_list:
        href = a['href']
        titleId = re.search('\?titleId=(\d+)', href)
        thumbnail = a.parent.select_one('img')['src']
        title = a.get_text(strip=True)
        cur_info = {title: titleId.group(1)}
        results.append(cur_info)
    
    # for문 순회가 끝난 후, 결과 dict가 저장된 리스트를 반환
    
    results2 = []
    
    for search_dict in results:
        for key, value in search_dict.items():
            results2.append(key)
            
    # results 중복 제거
    result2 = list(set(results2))
    
    print()
    
    for index, choose_list in enumerate(result2):
        print(f'{index+1}''.',choose_list)
        
    print('\n0. 이전')
    value = input('\n웹툰 선택: ')

    if value == '0':
        menu_move(menu_value)
    else:
        search = result2[int(value)-1]
        
        for item in results:
            for key, value in item.items():
                if search == key:
                    id = value
        
        if menu_value == '1':
            webtoon_detail(search, id)
        elif menu_value == '2':
            episode()
        
def webtoon_detail(title, title_id):
    # title_id에 해당하는 웹툰의 상세정보를 리턴해준다
    print('\n<<<정보(',title,')>>>')
    url = f'https://comic.naver.com/webtoon/list.nhn?titleId={title_id}'
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html)
    
    div_comicinfo = soup.select_one('div.comicinfo')
    div_detail = div_comicinfo.select_one('div.detail')
    
    title = div_detail.select_one('h2').contents[0].strip()
    author = div_detail.select_one('span.wrt_nm').get_text(strip=True)
    description = div_detail.select_one('p').get_text('\n', strip=True)
    
    # genre, age
    div_detail_info = div_detail.select_one('p.detail_info')
    genre = div_detail_info.select_one('span.genre').get_text(strip=True)
    age = div_detail_info.select_one('span.age').get_text(strip=True)
    
    # 모든 로직이 이 함수 안에 전부 존재
    print('제목:', title)
    print('설명:', description)
    print('작가:', author)
    print('장르:', [g.strip() for g in genre.split(',')])
    print('연령:', age)
    
    print()
    menu()
    
def episode():
    print('episode')
    menu()

# 초기 시작
menu()

<<웹툰 크롤러>>

    1. 정보
    2. 에피소드 목록
    3. 나가기
    
